Setup

In [0]:

%pip install -U -qqq langchain_core langchain_databricks langchain_community
%restart_python

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.69.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector databricks-sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.4 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/733.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.7/733.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.2 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/792.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 46.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
  Attempting uninstall: databricks-sdk
    Found existing installation: databricks-sdk 0.40.0
    Not uninstalling databricks-sdk at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pytho

In [0]:
dbutils.library.restartPython()

In [0]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from databricks_langchain import ChatDatabricks

db = SQLDatabase.from_databricks(catalog="accessmatelocal", schema="hotel", engine_args={"pool_pre_ping": True})
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-405b-instruct",
    temperature=0.1,
    max_tokens=250,
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

In [0]:
# agent.run(input={"query":"Use the table booking_hotel_listings and column most_popular_facilities and give me the hotel names of the hotels that have 'Facilities for disabled guests' in column most_popular_facilities"}, handle_parsing_errors=True)

# agent.run(input={"query":"Give me the hotel names of the hotels that have 'Facilities for disabled guests' in column most_popular_facilities"}, handle_parsing_errors=True)

agent.run(input={"query":"Give me the names of the hotels in London with swimming pool"}, handle_parsing_errors=True)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: booking_hotel_listingAction: sql_db_schema
Action Input: booking_hotel_listing
CREATE TABLE booking_hotel_listing (
	availability STRING, 
	city STRING, 
	coordinates STRING, 
	country STRING, 
	description STRING, 
	fine_print STRING, 
	hotel_id STRING, 
	house_rules STRING, 
	images STRING, 
	location STRING, 
	managed_by STRING, 
	manager_language_spoken STRING, 
	manager_score DOUBLE, 
	metro_railway_access BOOLEAN, 
	most_popular_facilities STRING, 
	number_of_reviews BIGINT, 
	popular_facilities STRING, 
	property_highlights STRING, 
	property_information STRING, 
	property_surroundings STRING, 
	review_score DOUBLE, 
	reviews_scores STRING, 
	timestamp STRING, 
	title STRING, 
	top_reviews STRING, 
	url STRING, 
	has_disabled_facilities BOOLEAN
) USING DELTA
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'enabled')

/*
3 rows from booking_hotel_listing table:
availability	city	coordinates	

'The names of the hotels in London with a swimming pool are [list of hotel titles returned by the query].'